Import Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Reading Dataset

In [ ]:
# Load movie data
movies_data = pd.read_csv('movies.csv')

Handling Null Values

Also converting values to lower string for convienice

In [ ]:
selected_features = ['genres', 'title', 'cast', 'release_date']
for feature in selected_features:
    if feature != 'release_date':
        movies_data[feature] = movies_data[feature].fillna(' ').str.lower()

Getting the Movie Name

In [ ]:
movieName = input("Enter movie name: ").lower()


# Filter DataFrame based on user input movie name
movie_row = movies_data[movies_data['title'] == movieName]

Recommending Movies

In [ ]:
# Check if movie exists in the database
if not movie_row.empty:
    # Retrieve genre and cast for the input movie
    userGenres = movie_row['genres'].iloc[0]
    userCast = movie_row['cast'].iloc[0]

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(movies_data['genres'] + ' ' + movies_data['cast'])

    # Compute cosine similarity between input movie and all other movies
    cosine_sim = cosine_similarity(tfidf_matrix[movie_row.index], tfidf_matrix)

    # Get indices of movies with highest similarity scores
    similar_movies_indices = cosine_sim.argsort()[0][::-1]

    # Print recommended movies
    print("Recommended movies similar to '{}' based on genres and cast:".format(movieName.capitalize()))
    for index, a in enumerate(similar_movies_indices[1:31], start=1):  # Exclude the input movie itself
        print(f"{index}. {movies_data['title'].iloc[a]}")

else:
    print("Movie '{}' not found.".format(movieName.capitalize()))

    # Offer user options
    print("\nWhat would you like to do next?")
    print("1. Explore movies in a specific genre.")
    print("2. Get random movie recommendations.")
    choice = input("Enter your choice (1 or 2): ")

    if choice == '1':
        # Offer options to explore genres
        print("\nEnter the genre you're interested in:")
        user_genre = input().lower().strip()

        # Filter movies by user entered genre
        genre_filtered_movies = movies_data[movies_data['genres'].str.contains(user_genre)]
        if not genre_filtered_movies.empty:
            print("\nHere are some movies related to the genre '{}' for you:".format(user_genre))
            print(genre_filtered_movies['title'].sample(10).to_string(index=False))  # Display 10 random movies
        else:
            print("No movies found for the genre '{}'.".format(user_genre))

    elif choice == '2':
        # Offer random recommendations
        print("\nHere are some random movie recommendations for you:")
        print(movies_data['title'].sample(10).to_string(index=False))  # Display 10 random movies

    else:
        print("Invalid choice. Please try again.")